In [1]:
# Importar librerias:
from shapely.geometry import Point, LineString
import pandas as pd
import geopandas as gpd
import os
print('Realizado')

Realizado


### Lectura de archivos

In [8]:
original = 'data/original.csv'
desplazado = 'data/desplazado.csv'
print('Realizado')

Realizado


### Convertir a DataFrame

In [9]:
dfOrg = pd.read_csv(original, sep=';')
dfDpz = pd.read_csv(desplazado, sep=';')
print('Realizado')

Realizado


In [10]:
dfOrg.head(3)

,id,lon,lat
0,32,-73.042052,-3.499631
1,33,-73.139910,-3.588879
2,34,-72.764743,-3.422353


In [11]:
dfDpz.head(3)

,id,lon,lat
0,32,-73.040750,-3.502746
1,33,-73.138740,-3.587896
2,34,-72.765599,-3.423195


### Unir los archivos por el campo id

In [13]:
# Merge: Unir dataframes
dfMerge = dfOrg.merge(dfDpz, on='id', how='left', suffixes=('_org', '_dpz'))

# Visualizar resultados
dfMerge.head()

,id,lon_org,lat_org,lon_dpz,lat_dpz
0,32,-73.042052,-3.499631,-73.040750,-3.502746
1,33,-73.139910,-3.588879,-73.138740,-3.587896
2,34,-72.764743,-3.422353,-72.765599,-3.423195
3,35,-72.255437,-3.587455,-72.256289,-3.584875
4,36,-73.091047,-3.498959,-73.093628,-3.503485


### Función para crear lineString a partir de dos pares de coordenadas:

In [15]:
funLineString = lambda lon_org, lat_org, lon_dpz, lat_dpz: LineString([(lon_org, lat_org), (lon_dpz, lat_dpz)])
print('Realizado')

Realizado


### Aplicar la función al dataframe

In [21]:
dfMerge['geometry'] = dfMerge.apply(lambda row: funLineString(row['lon_org'],
                                                              row['lat_org'],
                                                              row['lon_dpz'], 
                                                              row['lat_dpz']
                                                             ),                                    
                                    axis=1)
print('Realizado')

Realizado


In [22]:
dfMerge.head()

,id,lon_org,lat_org,lon_dpz,lat_dpz,geometry
0,32,-73.042052,-3.499631,-73.040750,-3.502746,"LINESTRING (-73.042052 -3.499631, -73.04075 -3..."
1,33,-73.139910,-3.588879,-73.138740,-3.587896,"LINESTRING (-73.13991 -3.588879, -73.13874 -3...."
2,34,-72.764743,-3.422353,-72.765599,-3.423195,"LINESTRING (-72.764743 -3.422353, -72.76559899..."
3,35,-72.255437,-3.587455,-72.256289,-3.584875,"LINESTRING (-72.255437 -3.587455, -72.256289 -..."
4,36,-73.091047,-3.498959,-73.093628,-3.503485,"LINESTRING (-73.091047 -3.498959, -73.093628 -..."


### Convertir dataframe a geodataframe

In [31]:
geoDf = gpd.GeoDataFrame(dfMerge, geometry=dfMerge.geometry, crs='EPSG:4326')
print('Realizado')

Realizado


In [32]:
geoDf.head()

,id,lon_org,lat_org,lon_dpz,lat_dpz,geometry
0,32,-73.042052,-3.499631,-73.040750,-3.502746,"LINESTRING (-73.04205 -3.49963, -73.04075 -3.5..."
1,33,-73.139910,-3.588879,-73.138740,-3.587896,"LINESTRING (-73.13991 -3.58888, -73.13874 -3.5..."
2,34,-72.764743,-3.422353,-72.765599,-3.423195,"LINESTRING (-72.76474 -3.42235, -72.76560 -3.4..."
3,35,-72.255437,-3.587455,-72.256289,-3.584875,"LINESTRING (-72.25544 -3.58745, -72.25629 -3.5..."
4,36,-73.091047,-3.498959,-73.093628,-3.503485,"LINESTRING (-73.09105 -3.49896, -73.09363 -3.5..."


### Calcular la longitud (distancia)

In [35]:
geoDf['distancia'] = geoDf.geometry.to_crs(32718).length
print('Realizado')

Realizado


In [36]:
geoDf.head()

,id,lon_org,lat_org,lon_dpz,lat_dpz,geometry,distancia
0,32,-73.042052,-3.499631,-73.040750,-3.502746,"LINESTRING (-73.04205 -3.49963, -73.04075 -3.5...",373.668514
1,33,-73.139910,-3.588879,-73.138740,-3.587896,"LINESTRING (-73.13991 -3.58888, -73.13874 -3.5...",169.470541
2,34,-72.764743,-3.422353,-72.765599,-3.423195,"LINESTRING (-72.76474 -3.42235, -72.76560 -3.4...",133.152919
3,35,-72.255437,-3.587455,-72.256289,-3.584875,"LINESTRING (-72.25544 -3.58745, -72.25629 -3.5...",300.812541
4,36,-73.091047,-3.498959,-73.093628,-3.503485,"LINESTRING (-73.09105 -3.49896, -73.09363 -3.5...",576.911058


### Exportar resultados

In [38]:
geoDf.to_file('out/gis.gpkg', layer='line_distance', driver='GPKG')
print('Realizado')

Realizado
